## proxy

In [1]:
# PROXY
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

## Dataset


In [2]:
from datasets import load_dataset
datas = load_dataset("bigbio/biored")

Using the latest cached version of the module from /root/.cache/huggingface/modules/datasets_modules/datasets/bigbio--biored/18063bd2d68db98142aaf46df1a6436e066ae226b6d0fbe293bd452e7bcdce23 (last modified on Mon Mar 18 01:26:54 2024) since it couldn't be found locally at bigbio/biored, or remotely on the Hugging Face Hub.
Repo card metadata block was not found. Setting CardData to empty.


## Extract

In [3]:
import json
import os, sys
import contextlib
from text2knowledge.strategy1 import extract_concepts


model_name = 'mistral:latest'

i = 4
while i < len(datas['test']):
    data = datas['test'][i]
    pmid = data['pmid']
    passages = data['passages']
    entities = data['entities']
    abstract = passages[1]['text'][0]
    abstract_offset = passages[1]['offsets'][0]
    metadata = {
        'pmid': pmid,
        'type': 'abstract',
    }

    #with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    entities_extracted = extract_concepts(abstract, model=model_name, metadata=metadata)
    
    if entities_extracted:
        disease_entities_extracted = [e for e in entities_extracted if e['category'] == 'Disease']
        # print(disease_entities_extracted)
        entities_str = json.dumps(entities_extracted, indent=4)
        # print(entities_str)
    else:
        print("No entities found.")
    break

 [
    {
        "entity": "mitochondrial DNA depletion",
        "confidence": "5",
        "category": "Metabolite"
    },
    {
        "entity": "hepatocerebral phenotype",
        "confidence": "4",
        "category": "Symptom"
    },
    {
        "entity": "deoxyguanosine kinase (dGK)",
        "confidence": "5",
        "category": "Enzyme (Protein)"
    },
    {
        "entity": "DGUOK gene",
        "confidence": "4",
        "category": "Gene"
    },
    {
        "entity": "combined respiratory chain deficiencies",
        "confidence": "4",
        "category": "Symptom"
    },
    {
        "entity": "dGK kinase domain",
        "confidence": "4",
        "category": "ProteinDomain"
    },
    {
        "entity": "North-African population",
        "confidence": "3",
        "category": "Population"
    },
    {
        "entity": "c.444-62C>A (mutation)",
        "confidence": "5",
        "category": "Mutation"
    },
    {
        "entity": "intra3 (intron 3)",
       

In [4]:
entities_extracted
'''
[
    {
        'entity': str,
        'confidence': str,
        'category': str,
        'pmid': str,
        'type': 'abstract'
    },
    ...
]
'''

[{'entity': 'mitochondrial DNA depletion',
  'confidence': '5',
  'category': 'Metabolite',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'hepatocerebral phenotype',
  'confidence': '4',
  'category': 'Symptom',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'deoxyguanosine kinase (dGK)',
  'confidence': '5',
  'category': 'Enzyme (Protein)',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'DGUOK gene',
  'confidence': '4',
  'category': 'Gene',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'combined respiratory chain deficiencies',
  'confidence': '4',
  'category': 'Symptom',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'dGK kinase domain',
  'confidence': '4',
  'category': 'ProteinDomain',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'North-African population',
  'confidence': '3',
  'category': 'Population',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'c.444-62C>A (mutation)',
  'confidence': '5',
  'cat

{'type': 'abstract',
 'text': ['Deoxyguanosine kinase (dGK) deficiency is a frequent cause of mitochondrial DNA depletion associated with a hepatocerebral phenotype. In this study, we describe a new splice site mutation in the DGUOK gene and the clinical, radiologic, and genetic features of these DGUOK patients. This new DGUOK homozygous mutation (c.444-62C>A) was identified in three patients from two North-African consanguineous families with combined respiratory chain deficiencies and mitochondrial DNA depletion in the liver. Brain MRIs are normal in DGUOK patients in the literature. Interestingly, we found subtentorial abnormal myelination and moderate hyperintensity in the bilateral pallidi in our patients. This new mutation creates a cryptic splice site in intron 3 (in position -62) and is predicted to result in a larger protein with an in-frame insertion of 20 amino acids. In silico analysis of the putative impact of the insertion shows serious clashes in protein conformation: th

In [17]:
abstract_offset = data['passages'][1]['offsets'][0]

def inc(r1, r2):
    # print(r1, r2)
    if r1[0] >= r2[0] and r1[1] <= r2[1]:
        return True
    return False

entities_abstract = [entity for entity in data['entities'] if inc(entity['offsets'][0], abstract_offset)]

In [18]:
entities_abstract

[{'text': ['Deoxyguanosine kinase (dGK) deficiency'],
  'offsets': [[102, 140]],
  'concept_id': 'C580039',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['mitochondrial DNA depletion'],
  'offsets': [[164, 191]],
  'concept_id': 'C580039',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['DGUOK'],
  'offsets': [[297, 302]],
  'concept_id': '1716',
  'semantic_type_id': 'GeneOrGeneProduct'},
 {'text': ['DGUOK'],
  'offsets': [[368, 373]],
  'concept_id': '1716',
  'semantic_type_id': 'GeneOrGeneProduct'},
 {'text': ['patients'],
  'offsets': [[374, 382]],
  'concept_id': '9606',
  'semantic_type_id': 'OrganismTaxon'},
 {'text': ['DGUOK'],
  'offsets': [[393, 398]],
  'concept_id': '1716',
  'semantic_type_id': 'GeneOrGeneProduct'},
 {'text': ['c.444-62C>A'],
  'offsets': [[420, 431]],
  'concept_id': 'c|SUB|C|444-62|A',
  'semantic_type_id': 'SequenceVariant'},
 {'text': ['patients'],
  'offsets': [[457, 465]],
  'concept_id': '9606',
  'semantic_type_id

## Normalizer

In [19]:
from text2knowledge.normalizer import Normalizer

class Arg:
    def __init__(self, model_name_or_path, dictionary_path):
        self.model_name_or_path = model_name_or_path
        self.show_embeddings = False
        self.show_predictions = True
        self.dictionary_path = dictionary_path
        self.use_cuda = True

In [24]:
# 'Gene', 'Protein', 'Compound', 'Disease', 'Symptom', 'Pathway', 'Anatomy', 'Metabolite', 'MolecularFunction', 'BiologicalProcess', 'CellularComponent'

# Disease
# arg_d = Arg("dmis-lab/biosyn-biobert-ncbi-disease", "/root/text2knowledge/data/dictionary/yang/dict_Disease.txt")
arg_d = Arg("dmis-lab/biosyn-biobert-ncbi-disease", "/root/text2knowledge/data/dictionary/dict_Disease_20210630.txt")

# Gene
# arg_g = Arg("dmis-lab/biosyn-biobert-ncbi-disease", "/root/text2knowledge/data/dictionary/dict_Disease_20210630.txt")

In [25]:
disease_normalizer = Normalizer(arg_d)

/root/miniconda3/lib/python3.8/site-packages/huggingface_hub/file_download.py:678: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainabi

In [26]:
disease_normalizer.normalize("long QT syndrome (LQTS)")

/root/miniconda3/lib/python3.8/site-packages/biosyn-0.1-py3.8.egg/biosyn/biosyn.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Loaded dictionary from cached file ./tmp/cached_biosyn-biobert-ncbi-disease_dict_Disease_20210630.pk


{'mention': 'long qt syndrome lqts',
 'predictions': [{'name': 'Long QT Syndrome', 'id': 'MESH:D008133'},
  {'name': 'Long Qt Syndrome 1/2', 'id': 'MESH:C566006'},
  {'name': 'Long Qt Syndrome 2|LONG QT SYNDROME 1/2, DIGENIC, INCLUDED;LQT1/2, DIGENIC, INCLUDED|LONG QT SYNDROME 2/3, DIGENIC, INCLUDED;LQT2/3, DIGENIC, INCLUDED|LONG QT SYNDROME 2/5, DIGENIC, INCLUDED;LQT2/5, DIGENIC, INCLUDED|LONG QT SYNDROME 2/9, DIGENIC, INCLUDED;LQT2/9, DIGENIC, INCLUDED|LONG QT SYNDROME 2, ACQUIRED, SUSCEPTIBILITY TO, INCLUDED|LQT2',
   'id': 'MESH:C563614'},
  {'name': 'Long Qt Syndrome 10', 'id': 'OMIM:611819,MESH:C567514'},
  {'name': 'LONG QT SYNDROME 13', 'id': 'OMIM:613485'}]}

In [23]:
disease_normalizer.normalize("long QT syndrome")

Loaded dictionary from cached file ./tmp/cached_biosyn-biobert-ncbi-disease_dict_Disease.pk


{'mention': 'long qt syndrome',
 'predictions': [{'name': 'Long Qt Syndrome 1-2', 'id': 'MESH:C566006'},
  {'name': 'Long QT syndrome type 3', 'id': 'MESH:C537034'},
  {'name': 'obsolete long QT syndrome 1', 'id': 'MONDO:0008646'},
  {'name': 'Long Qt Syndrome 3-6', 'id': 'MESH:C566334'},
  {'name': 'Long Qt Syndrome 2-3', 'id': 'MESH:C565841'}]}

In [ ]:
def normalize(name, category):
    if category not in ['Gene', 'Protein', 'Compound', 'Disease', 'Symptom', 'Pathway', 'Anatomy', 'Metabolite', 'MolecularFunction', 'BiologicalProcess', 'CellularComponent']:
        raise ValueError('Wrong category')
    pass

In [ ]:
scores = []
for data in datas['test']:
    pmid = data['pmid']
    passages = data['passages']
    entities = data['entities']
    abstract = passages[1]['text'][0]
    abstract_offset = passages[1]['offsets'][0]
    metadata = {
        'pmid': pmid,
        'type': 'abstract',
    }

    score = 0
    #with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    entities_extracted = extract_concepts(abstract, model=model_name, metadata=metadata)
    
    
    if entities_extracted:
        # normalize extracted entities to DB
        entities_extracted_normalized = []
        for entity in entities_extracted:
            name = entity['entity']
            category = entity['category']
            ID = normalize(name, category) # TODO
            entity['DBID'] = ID
            entities_extracted_normalized.append(entity)

        # normalize answer to DB as well in order to calc score later
        entities_ans_normalized = []
        for entity in entities:
            name = entity['text'][0]
            category = entity['semantic_type_id']
            ID = normalize(name, category) # TODO
            entity['DBID'] = ID
            entities_ans_normalized.append(entity)

        score = calc_score(entities_extracted_normalized, entities_ans_normalized, abstract, abstract_offset)  # TODO
        scores.append(score)

In [33]:
data

{'pmid': '19394258',
 'passages': [{'type': 'title',
   'text': ['The first founder DGUOK mutation associated with hepatocerebral mitochondrial DNA depletion syndrome.'],
   'offsets': [[0, 101]]},
  {'type': 'abstract',
   'text': ['Deoxyguanosine kinase (dGK) deficiency is a frequent cause of mitochondrial DNA depletion associated with a hepatocerebral phenotype. In this study, we describe a new splice site mutation in the DGUOK gene and the clinical, radiologic, and genetic features of these DGUOK patients. This new DGUOK homozygous mutation (c.444-62C>A) was identified in three patients from two North-African consanguineous families with combined respiratory chain deficiencies and mitochondrial DNA depletion in the liver. Brain MRIs are normal in DGUOK patients in the literature. Interestingly, we found subtentorial abnormal myelination and moderate hyperintensity in the bilateral pallidi in our patients. This new mutation creates a cryptic splice site in intron 3 (in position -62)

In [45]:
def find_substring_indices(text, substring):
    indices = []
    index = text.find(substring)
    while index != -1:
        indices.append(index)
        index = text.find(substring, index + 1)
    return indices

def overlap(r1, r2):
    if r1[1] >= r2[0] and r1[0] <= r2[1]:
        return True
    return False

def calc_score(extracted, gt, text, offset):
    # get occurance of extracted entities in text
    for entity in extracted:
        name = entity['entity']
        occur = [[i+offset, i+offset+len(name)] for i in find_substring_indices(text, name)]
        entity['offsets'] = occur
    COR = 0
    INC = 0
    MIS = 0
    SPU = 0
    for entity in gt:
        is_COR = False
        is_MIS = True
        loc = entity['offsets'][0]
        for e in extracted:
            for o in e['offsets']:
                if overlap(loc, o):
                    is_MIS = False
                    if e['DBID'] == entity['DBID']:
                        is_COR = True
        if is_MIS:
            MIS += 1
        elif is_COR:
            COR += 1
        else:
            INC += 1
    for entity in extracted:
        is_SPU = True
        locs = entity['offsets']
        for o in loc:
            for e in gt:
                if overlap(o, e['offsets'][0]):
                    is_SPU = False
        if is_SPU:
            SPU += 1
    POS = COR + INC + MIS
    ACT = COR + INC + SPU
    P = COR / ACT
    R = COR / POS
    F1 = 2 * P * R / (P + R)
    return F1

In [46]:
calc_score(entities_extracted, data['entities'], data['passages'][1]['text'][0], data['passages'][1]['offsets'][0][0])

KeyError: 'DBID'

In [38]:
entities_extracted

[{'entity': 'mitochondrial DNA depletion',
  'confidence': '5',
  'category': 'Metabolite',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'hepatocerebral phenotype',
  'confidence': '4',
  'category': 'Symptom',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'deoxyguanosine kinase (dGK)',
  'confidence': '5',
  'category': 'Enzyme (Protein)',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'DGUOK gene',
  'confidence': '4',
  'category': 'Gene',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'combined respiratory chain deficiencies',
  'confidence': '4',
  'category': 'Symptom',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'dGK kinase domain',
  'confidence': '4',
  'category': 'ProteinDomain',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'North-African population',
  'confidence': '3',
  'category': 'Population',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'c.444-62C>A (mutation)',
  'confidence': '5',
  'cat

In [35]:
calc_score

True

[0, 3, 6, 28]